<a href="https://colab.research.google.com/github/RSvan/ProteinMPNN_notebooks/blob/main/notebooks/ProteinMPNN_subset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ProteinMPNN_subset
This notebook is will clone the ProteinMPNN repository and allow for selective residue design of an input protein structure. The majority of the steps are via the command line. This also allows for running other protocols within this notebook. The results will be packaged and downloaded. There is an option to disallow the native amino acid during design.

In [ ]:
#@title Install ProteinMPNN from github + additional functions
import json, time, os, sys, glob

if not os.path.isdir("ProteinMPNN"):
  os.system("git clone -q https://github.com/dauparas/ProteinMPNN.git")
sys.path.append('/content/ProteinMPNN')

os.system('pip install biopython')
os.system('pip install itables')

import numpy as np
import subprocess
from google.colab import files
import datetime
import Bio.PDB as BPDB
from Bio.SeqUtils import seq1
import pandas as pd
from itables import init_notebook_mode, show
import ipywidgets as widgets

def bias_input_aa(output_path, chain_list, position_list):
  mpnn_alphabet = 'ACDEFGHIKLMNPQRSTVWYX'

  mpnn_alphabet_dict = {'A': 0,'C': 1,'D': 2,'E': 3,'F': 4,'G': 5,'H': 6,'I': 7,'K': 8,'L': 9,'M': 10,'N': 11,'P': 12,'Q': 13,'R': 14,'S': 15,'T': 16,'V': 17,'W': 18,'Y': 19,'X': 20}

  with open(path_for_parsed_chains, 'r') as json_file:
    json_list = list(json_file)

  if chain_list != '':
    global_designed_chain_list = [str(item).strip() for item in chain_list.split()]
  design_list = [[int(item)-1 for item in one.split()] for one in position_list.split(',')]

  my_dict = {}
  for json_str in json_list:
    result = json.loads(json_str)
    all_chain_list = [item[-1:] for item in list(result) if item[:10]=='seq_chain_']
    bias_by_res_dict = {}
    for chain in all_chain_list:
      chain_length = len(result[f'seq_chain_{chain}'])
      bias_per_residue = np.zeros([chain_length, 21])

      try:
        idx = global_designed_chain_list.index(chain)
        residues = design_list[idx]
        for res in residues:
          aa = mpnn_alphabet_dict[result['seq_chain_'+chain][res]]
          bias_per_residue[res, aa] = -1e6
      except:
        continue
      bias_by_res_dict[chain] = bias_per_residue.tolist()
    my_dict[result['name']] = bias_by_res_dict

  with open(output_path, 'w') as f:
    f.write(json.dumps(my_dict) + '\n')

def output_results(output_dir, base_name, no_residues, residue_prob_idx):
  alphabet = 'ACDEFGHIKLMNPQRSTVWYX'
  alphabet_w_commas = ', '.join(list(alphabet))
  output = np.load(output_dir + 'probs/' + base_name + '.npz')
  data_out = np.zeros((no_residues,21))
  probs_norm = np.zeros((no_residues,number_of_repeats,21))
  for n in range(no_residues):
    for m in range(number_of_repeats):
      probs_norm[n,m] = (output['probs'][m][int(residue_prob_idx[n])]/output['probs'][m][int(residue_prob_idx[n])].sum())*100
    data_out[n,:] = probs_norm[n,:].mean(axis=0)
  return data_out, probs_norm

alphabet = 'ACDEFGHIKLMNPQRSTVWYX'

In [ ]:
#@title Input pdb

#@markdown #### Inputs
pdb = '' #@param {type:'string'}
#@markdown - Leave blank to get an upload prompt
#@markdown - The header row of the table produced after the pdb is read denotes the number to use to define the residue position to design

if pdb is None or pdb == '':
  upload_dict = files.upload()
  pdb = list(upload_dict)[0]

if pdb[-3:] == 'pdb':
  dir_pdb = pdb[:-4]
else:
  dir_pdb = pdb
if not os.path.isdir('input_{}'.format(dir_pdb)):
  os.system('mkdir input_{}'.format(dir_pdb))
subprocess.run(['cp', pdb, 'input_{}/{}'.format(dir_pdb, pdb)])
if not os.path.isdir('output_{}'.format(dir_pdb)):
  os.system('mkdir output_{}'.format(dir_pdb))
output_dir = 'output_{}/'.format(dir_pdb)
input_dir = 'input_{}'.format(dir_pdb)

structure = BPDB.PDBParser(PERMISSIVE = True, QUIET = True).get_structure('curretnt',pdb)
# convert into dictionary for display
struct_dict = {chain.id:[[x.get_id()[1] for x in list(chain.get_residues())], ','.join(list(seq1(''.join(residue.resname for residue in chain))))] for chain in structure.get_chains()}
no_chains = len(struct_dict)
chain_ids = list(struct_dict.keys())
for m in range(no_chains):
  struct_dict[chain_ids[m]][0] = np.reshape(struct_dict[chain_ids[m]][0],(1,len(struct_dict[chain_ids[m]][0])))
pdb_df = pd.DataFrame()
idx = []
chain_lens = np.zeros((no_chains,1))
for m in range(no_chains):
  test_df = pd.DataFrame.from_records(struct_dict[chain_ids[m]][0])
  test_df.loc[1] = struct_dict[chain_ids[m]][1].split(',')
  pdb_df = pd.concat([pdb_df, test_df])
  idx.append(chain_ids[m]+'_res no')
  idx.append(chain_ids[m]+'_seq')
  chain_lens[m] = np.shape((struct_dict[chain_ids[m]][0]))[1]
pdb_df.index = idx
column_names_len = len(pdb_df.columns)
new_column_names = [str(x) for x in np.arange(1,column_names_len+1)]
pdb_df.columns = new_column_names

init_notebook_mode(all_interactive=True)

show(pdb_df, columnDefs=[{'orderable': False, 'targets': '_all' }])

In [ ]:
#@title Options

#@markdown #### Chain/residues
chains = 'A' #@param {type:'string'}
#@markdown - chains that will have residues modified separated by spaces (example: `A C`)
residue_positions = '10 11 12 13 15 ' #@param {type:'string'}
#@markdown - positions of resides to be modified (example: `1 3 10,2 4 5`)
#@markdown - These do not correspond to residue number - see the table above (top row) to determine position of the residues of interest

#@markdown #### Options
number_of_repeats = 8 #@param ['4', '8', '16', '32', '64', '128', '256', '512', '1024'] {type:"raw"}
sampling_temperature = 0.5 #@param ['0.0001', '0.1', '0.15', '0.2', '0.25', '0.3', '0.5', '0.7', '1.0'] {type:"raw"}
parent_residue_bias = False #@param {type:"boolean"}
#@markdown - Will modify the bias to eliminate modeling the input amino acids at the selected residues.

#@markdown #### Miscellaneous Parameters
random_seed = 36 #@param {type:'integer'}

# check for pdb and input values
proc_bool = 0
if os.path.isfile(pdb) or os.path.isfile(pdb+'.pdb'):
  if chains == '':
    print('Please input a chain or chains to be modified.')
  else:
    chains_to_design = chains
    if residue_positions == '':
      print('Please choose residues to mutate')
    else:
      design_only_positions = residue_positions
      seed = random_seed
      proc_bool = 1
else:
  print('Please upload a pdb for analysis.')

if proc_bool:
  residue_list= [[int(item) for item in one.split()] for one in residue_positions.split(',')]
  chain_list = chains.split()

  no_residues = len(design_only_positions.replace(',', ' ').split())
  residue_prob_idx = np.zeros((no_residues))
  residue_ident = []
  count = 0
  count_idx = 0
  for m in range(len(chain_list)):
    if chain_list[m] in chain_ids:
      for n in range(len(residue_list[m])):
        residue_prob_idx[count] = residue_list[m][n] + count_idx - 1
        residue_ident.append(chain_list[m]+str(pdb_df[str(residue_list[m][n])][chain_list[m]+'_res no'])+pdb_df[str(residue_list[m][n])][chain_list[m]+'_seq'])
        count += 1
      count_idx += chain_lens[m][0]

In [ ]:
#@title Run ProteinMPNN

if proc_bool == 1:
  folder_with_pdbs = input_dir
  path_for_parsed_chains = output_dir + 'parsed_pdbs.jsonl'
  path_for_assigned_chains = output_dir + 'assigned_pdbs.jsonl'
  path_for_fixed_positions = output_dir + 'fixed_pdbs.jsonl'
  date_object = datetime.date.today()
  if pdb[-3:] == 'pdb':
    filename_out = '{}_P-MPNN_{}_{}_{}_{}'.format(pdb[:-4],str(date_object),number_of_repeats,sampling_temperature,seed)
    base_name = pdb[:-4]
  else:
    filename_out = '{}_P-MPNN_{}_{}_{}_{}'.format(pdb,str(date_object),number_of_repeats,sampling_temperature,seed)
    base_name = pdb
  if parent_residue_bias:
    filename_out += '_Pbias'
  print('Parsing the PDB')
  parse_inpath = '--input_path=' + folder_with_pdbs
  parse_outpath = '--output_path=' + path_for_parsed_chains
  result = subprocess.run(['python', 'ProteinMPNN/helper_scripts/parse_multiple_chains.py', parse_inpath, parse_outpath])
  if result.returncode != 0:
    print('There was an error in parsing the pdb')
  print('Assigning chains to design')
  assign_inpath = '--input_path=' + path_for_parsed_chains
  assign_outpath = '--output_path=' + path_for_assigned_chains
  chain_list = '--chain_list=' + chains_to_design
  result = subprocess.run(['python', 'ProteinMPNN/helper_scripts/assign_fixed_chains.py', assign_inpath, assign_outpath, chain_list])
  if result.returncode != 0:
    print('There was an error in assigning chains to design')
  print('Assigning residues to design')
  design_inpath = '--input_path=' + path_for_parsed_chains
  design_outpath = '--output_path=' + path_for_fixed_positions
  chain_list = '--chain_list=' + chains_to_design
  position_list = '--position_list=' + design_only_positions
  result = subprocess.run(['python', 'ProteinMPNN/helper_scripts/make_fixed_positions_dict.py', design_inpath, design_outpath, chain_list, position_list, '--specify_non_fixed'])
  if result.returncode != 0:
    print('There was an error in assigning residues to design')
  jsonl_path = '--jsonl_path=' + path_for_parsed_chains
  chain_id = '--chain_id_jsonl=' + path_for_assigned_chains
  fixed_positions = '--fixed_positions_jsonl=' + path_for_fixed_positions
  out_folder = '--out_folder=' + output_dir
  num_seq = '--num_seq_per_target=' + str(number_of_repeats)
  samp_temp = '--sampling_temp=' + str(sampling_temperature)
  seed_in = '--seed=' + str(seed)
  batch = '--batch_size=' + '1'
  probs = '--save_probs=1'
  if parent_residue_bias:
    bias_input_aa('bias.jsonl', chains_to_design, design_only_positions)
    bias = '--bias_by_res_jsonl=' + 'bias.jsonl'
    print('Running design process with bias')
    result = subprocess.run(['python', 'ProteinMPNN/protein_mpnn_run.py', jsonl_path, chain_id, fixed_positions, out_folder, bias, num_seq, samp_temp, seed_in, batch, probs])
  else:
    print('Running design process')
    result = subprocess.run(['python', 'ProteinMPNN/protein_mpnn_run.py', jsonl_path, chain_id, fixed_positions, out_folder, num_seq, samp_temp, seed_in, batch, probs])
  if result.returncode != 0:
    print('There was an error during the design process')
  else:
    print('Tabulating and outputting probabilities')
    data_out, probs_out = output_results(output_dir, base_name, no_residues, residue_prob_idx)
    data_out_df = pd.DataFrame(data=data_out, index=residue_ident, columns=[x for x in alphabet]).map('{:.1f}'.format)
    print('Saving Output')
    os.system('zip -j {}.zip {}/seqs/{}.fa'.format(filename_out,output_dir,base_name))
    data_out_df.to_csv(filename_out+'.csv')
    compression_opts = dict(method='zip', archive_name=filename_out+'.csv')
    data_out_df.to_csv(filename_out+'.zip', mode='a', compression=compression_opts)
    for m in range(no_residues):
      res_data_out_df = pd.DataFrame(data=probs_out[m,:,:], index=[np.arange(1,number_of_repeats+1)], columns=[x for x in alphabet]).map('{:.1f}'.format)
      compression_opts = dict(method='zip', archive_name=filename_out+'_{}.csv'.format(residue_ident[m]))
      res_data_out_df.to_csv(filename_out+'.zip', mode='a', compression=compression_opts)
    print('Displaying Results')
    show(data_out_df, columnDefs=[{'orderable': False, 'targets': '_all' }])
    files.download(filename_out+'.csv')
    files.download(filename_out+'.zip')
else:
  print('Please correct any errors in the previous cell before proceeding')

In [ ]:
#@title Examine Individual Residues

def f(res_index):
  """show individual residues results"""
  res_data_out_df = pd.DataFrame(data=probs_out[res_index-1,:,:], index=[np.arange(1,number_of_repeats+1)], columns=[x for x in alphabet]).map('{:.1f}'.format)
  print(residue_ident[res_index-1])
  res_data_out_df.index.name = residue_ident[res_index-1]
  show(res_data_out_df, columnDefs=[{'orderable': False, 'targets': '_all' }])
widgets.interact(f, res_index = widgets.IntSlider(value=1, min=1, max=no_residues, step=1))